In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from Bio.Restriction import BsaI, BsmBI
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import warnings
warnings.filterwarnings("ignore")

In [2]:
def reindex_ps1(plasmid):
    '''Reindex plasmids to start from the annealing region of PS1 so the fragment will be in the middle of the sequence.
    This will do nothing for a fragment from pcr products as they do not have PS1 region'''
    
    new_start = plasmid.find('AGGGCGGCGGATTTGTCC')
    if new_start == -1:
        return plasmid
    return plasmid[new_start:] + plasmid[:new_start]

def len_amplicon(plasmid):
    '''Calculate the length of amplicon from PS1 to PS2, the plasmid always starts from PS1'''
    
    ps2 = 'GAACGCTCGGTTGCCGC' #reverse complement of PS2
    return len(plasmid[:plasmid.find(ps2)]) + len(ps2)

In [17]:
#import fasta files
fastas = ['split-mCherry-fragments.fasta', 'vectors.fasta']
plasmids = pd.DataFrame([(p.id, str(p.seq), fasta) for fasta in fastas \
                        for p in list(SeqIO.parse('datasets/jump/{}'.format(fasta), 'fasta'))], \
                        columns=['name', 'sequence', 'level'])
plasmids['level'] = plasmids['level'].str.split('.', expand=True)[0]
plasmids['sequence'] = plasmids['sequence'].str.upper()
plasmids['sequence'] = plasmids['sequence'].apply(reindex_ps1)
plasmids

,name,sequence,level
0,mCherry-gp41-1_C,CGTCTCGGTCTCAAATGCTGAAAAAAATCCTGAAAATCGAGGAACT...,split-mCherry-fragments
1,mCherry-gp41-1_Mut,CGTCTCGGTCTCAAATGGTGAGCAAGGGCGAAGAAGACAACATGGC...,split-mCherry-fragments
2,mCherry-gp41-1_N,CGTCTCGGTCTCAAATGGTGAGCAAGGGCGAAGAAGACAACATGGC...,split-mCherry-fragments
3,mCherry-gp41-8_C,CGTCTCGGTCTCAAATGTGTGAAATCTTTGAAAACGAGATCGACTG...,split-mCherry-fragments
4,mCherry-gp41-8_Mut,CGTCTCGGTCTCAAATGGTGAGCAAGGGCGAAGAAGACAACATGGC...,split-mCherry-fragments
5,mCherry-gp41-8_N,CGTCTCGGTCTCAAATGGTGAGCAAGGGCGAAGAAGACAACATGGC...,split-mCherry-fragments
6,mCherry-IMPDH-1_C,CGTCTCGGTCTCAAATGAAATTCAAACTGAAAGAGATCACCAGCAT...,split-mCherry-fragments
7,mCherry-IMPDH-1_Mut,CGTCTCGGTCTCAAATGGTGAGCAAGGGCGAAGAAGACAACATGGC...,split-mCherry-fragments
8,mCherry-IMPDH-1_N,CGTCTCGGTCTCAAATGGTGAGCAAGGGCGAAGAAGACAACATGGC...,split-mCherry-fragments
9,mCherry-M86_C,CGTCTCGGTCTCAAATGAGCTCCTCTTTACAATTGGCACCAGAAAT...,split-mCherry-fragments


In [18]:
plasmids['level'] = plasmids['level'].map({'split-mCherry-fragments': 'fragments', 'vectors': 'vectors'})

In [20]:
#map the name of vectors and parts obtained from addgenes
mapping = {
    'UAC': 'pJUMP18-Uac',
    '1A': 'pJUMP29-1A(sfGFP)',
    '1B': 'pJUMP29-1B(sfGFP)',
    '1B*': 'pJUMP29-1B*(sfGFP)',
    '1C': 'pJUMP29-1C(sfGFP)',
    '1C*': 'pJUMP29-1C*(sfGFP)',
    '1D\'': 'pJUMP29-1D\'(sfGFP)',
    '1D': 'pJUMP29-1D(sfGFP)',
    '1Ep': 'pJUMP29-1E\'(sfGFP)',
    '1E': 'pJUMP29-1E(sfGFP)',
    '1F': 'pJUMP29-1F(sfGFP)',
    '2A': 'pJUMP49-2A(sfGFP)',
    '2B': 'pJUMP49-2B(sfGFP)',
    '2B*': 'pJUMP49-2B*(sfGFP)',
    '2C': 'pJUMP49-2C(sfGFP)',
    '2C*': 'pJUMP49-2C*(sfGFP)',
    '2Dp': 'pJUMP49-2D\'(sfGFP)',
    '2D': 'pJUMP49-2D(sfGFP)',
    '2E': 'pJUMP49-2E(sfGFP)',
    'B0033_RN': 'pJUMP18-B0033-MV_RN',
    'B0033_R': 'pJUMP18-B0033_R',
    'B0034_RN': 'pJUMP18-B0034-MV_RN',
    'B0034_R': 'pJUMP18-B0034_R',
    'mCherry_O': 'pJUMP18-mCherry_O',
    'sGFP_O': 'pJUMP18-sfGFP_O',
    'B0015_CT': 'pJUMP19-B0015_CT',
    'B0015_T': 'pJUMP19-B0015_T',
    'J100_P': 'pJUMP19-23100_P',
}

In [21]:
def get_sites(part, odd_level=True, vector=False):
    
    enz = BsaI if odd_level else BsmBI
    
    if vector:
        
        site = str(Seq(enz.site).reverse_complement())
        right_idx = part.find(site) - 5
        right_cut = part[right_idx: right_idx+4]

        site = enz.site
        left_idx = part.find(site) + len(site) + 1
        left_cut = part[left_idx: left_idx+4]
        
        fragment = part[left_idx:] + part[:right_idx+4]
        
    else:
        site = enz.site
        left_idx = part.find(site) + len(site) + 1
        left_cut = part[left_idx: left_idx+4]
        
        site = str(Seq(enz.site).reverse_complement())
        right_idx = part.find(site) - 5
        right_cut = part[right_idx: right_idx+4]
        
        fragment = part[left_idx: right_idx+4]
        
    return left_cut, right_cut, fragment

#part_name_u, get_sites(part_seq_u, odd_level=False, vector=True)

Level 0.

In [22]:
fragments = plasmids[plasmids['level']=='fragments']
fragments_map = dict(zip(fragments['name'], fragments['sequence']))
vectors = plasmids[plasmids['level']=='vectors']
vectors_map = dict(zip(vectors['name'], vectors['sequence']))
uac = vectors_map[mapping['UAC']]
uac

'AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACAACAGATAAAACGAAAGGCCCAGTCTTTCGACTGAGCCTTTCGTTTTATTTGATGCCTTTAATTAAGGAGTTTTGCAGGTGCACCTGCTTTTCGCTGAATTCGCGGCCGCTTCTAGAGGGTCTGCGATGTTTGGTCTTGAGACGACTGTGACAAGGAGTTGACGGCTAGCTCAGTCCTAGGTACAGTGCTAGCGTACTTGTTTAACTTTAAGAAGGAGATATACAATGGTAGCCCGTAAAGGCGAAGAGCTGTTCACTGGTGTCGTCCCTATTCTGGTGGAACTGGATGGTGATGTCAACGGTCATAAGTTTTCCGTGCGTGGCGAGGGTGAAGGTGACGCAACTAATGGTAAACTGACGCTGAAGTTCATCTGTACTACTGGTAAACTGCCGGTACCTTGGCCGACTCTGGTAACGACGCTGACTTATGGTGTTCAGTGCTTTGCTCGTTATCCGGACCATATGAAGCAGCATGACTTCTTCAAGTCCGCCATGCCGGAAGGCTATGTGCAGGAACGCACGATTTCCTTTAAGGATGACGGCACGTACAAAACGCGTGCGGAAGTGAAATTTGAAGGCGATACCCTGGTAAACCGCATTGAGCTGAAAGGCATTGACTTTAAAGAAGATGGCAATATCCTGGGCCATAAGCTGGAATACAATTTTAACAGCCACAATGTTTACATCACCGCCGATAAACAAAAAAATGGCATTAAAGCGAATTTTAAAATTCGCCACAACGTGGAGGATGGCAGCGTGCAGCTGGCTGATCACTACCAGCAAAACACTCCAATCGGTGATGGTCCTGTTCTGCTGCCAGACAATCACTATCTGAGCACGCAAAGCGTTCTGTCTAAAGATCCGAACGAGAAACGCGATCATATGGTTCTGCTGGAGTTCGTAACCGCAGCGGGCATCACGCATGGTATGGATGAACTGTACAAAGGTTCGTAA

In [27]:
fragments = plasmids[plasmids['level']=='fragments']
fragments_map = dict(zip(fragments['name'], fragments['sequence']))
vectors = plasmids[plasmids['level']=='vectors']
vectors_map = dict(zip(vectors['name'], vectors['sequence']))
uac = vectors_map[mapping['UAC']]

def assemble_lvl_0(fragments, uac):
    
    constructs = []
    for name in fragments:

        uac_sites = get_sites(uac, odd_level=False, vector=True)
        frag_sites = get_sites(fragments[name], odd_level=False, vector=False)
        print(uac_sites, frag_sites)
        
        if (uac_sites[1]==frag_sites[0] and uac_sites[0]==frag_sites[1]):
            assembly = reindex_ps1(frag_sites[2][:-4] + uac_sites[2][:-4])
            constructs.append((name, assembly))
            
    return pd.DataFrame(constructs, columns=['name', 'sequence'])

lvl0 = assemble_lvl_0(fragments_map, uac)
#new_promoters = lvl0[lvl0['name'].isin(['PBAD-RiboJ', 'PLuxB-RiboJ', 'PSalTTC-RiboJ', 'PBetI-RiboJ', 'PTac-RiboJ'])]
lvl0['name'] = lvl0['name'] + '_NOC'
lvl0['size'] = lvl0['sequence'].apply(lambda x: len(x))
lvl0.to_csv('datasets/jump/level-0-mCherry.csv', index=False)
#new_promoters
lvl0

('GACC', 'GTCT', 'GACCTTTCATCGCGACCTACTAGTAGCGGCCGCTGCAGGGAGTTGTCTTCGAAGACTTCGCTCTAGTCTTGGACTCCTGTTGATAGATCCAGTAATGACCTCAGAACTCCATCTGGATTTGTTCAGAACGCTCGGTTGCCGCCGGGCGTTTTTTATTGGTGAGAATCCAGGGGTCCCCAATAATTACGATTTAAATTAGTAGCCCGCCTAATGAGCGGGCTTTTTTTTAATTCCCCTATTTGTTTATTTTTCTAAATACATTCAAATATGTATCCGCTCATGAGACAATAACCCTGATAAATGCTTCAATAATATTGAAAAAGGAAGAGTATGAGCATTCAGCATTTTCGTGTGGCGCTGATTCCGTTTTTTGCGGCGTTTTGCCTGCCGGTGTTTGCGCATCCGGAAACCCTGGTGAAAGTGAAAGATGCGGAAGATCAACTGGGTGCGCGCGTGGGCTATATTGAACTGGATCTGAACAGCGGCAAAATTCTGGAATCTTTTCGTCCGGAAGAACGTTTTCCGATGATGAGCACCTTTAAAGTGCTGCTGTGCGGTGCGGTTCTGAGCCGTGTGGATGCGGGCCAGGAACAACTGGGCCGTCGTATTCATTATAGCCAGAACGATCTGGTGGAATATAGCCCGGTGACCGAAAAACATCTGACCGATGGCATGACCGTGCGTGAACTGTGCAGCGCGGCGATTACCATGAGCGATAACACCGCGGCGAACCTGCTGCTGACGACCATTGGCGGTCCGAAAGAACTGACCGCGTTTCTGCATAACATGGGCGATCATGTGACCCGTCTGGATCGTTGGGAACCGGAACTGAACGAAGCGATTCCGAACGATGAACGTGATACCACCATGCCGGCAGCAATGGCGACCACCCTGCGTAAACTGCTGACGGGTGAGCTGCTGACCCTGGCAAGCCGCCAGCAACTGATTGATTGGATGGAAGCGGATAAAGTGGCGGGTCCGC

,name,sequence,size
0,mCherry-gp41-1_C_NOC,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,2991
1,mCherry-gp41-1_Mut_NOC,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,3369
2,mCherry-gp41-1_N_NOC,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,3393
3,mCherry-gp41-8_C_NOC,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,3018
4,mCherry-gp41-8_Mut_NOC,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,3369
5,mCherry-gp41-8_N_NOC,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,3396
6,mCherry-IMPDH-1_C_NOC,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,3003
7,mCherry-IMPDH-1_Mut_NOC,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,3369
8,mCherry-IMPDH-1_N_NOC,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,3432
9,mCherry-M86_C_NOC,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,3048


In [ ]:
get_sites('CGTCTCGGTCTCAGGAGACCTGTAGGATCGTACAGGTTTACGCAAGAAAATGGTTTGTTACAGTCGAATAAATACTTGAGACCGAGACG', odd_level=False, vector=False)

Level 1.

In [7]:
assembly_plan = pd.read_csv('datasets/jump/level-1-assembly.csv')
lvl1_assembly = assembly_plan.iloc[:, 2:].melt().drop_duplicates().reset_index(drop=True)
lvl1_assembly['name'] = lvl1_assembly['value'].map(mapping)
#lvl1_assembly['name'] = lvl1_assembly['value'].apply(lambda x: mapping[x] if x in mapping else '')
lvl1_assembly.loc[lvl1_assembly['name'].isnull(), 'name'] = 'pJ0-' + lvl1_assembly['value']
#lvl1_assembly = lvl1_assembly
lvl1_assembly = pd.merge(lvl1_assembly, plasmids, on='name', how='left')
assembly_plan_map = dict(zip(lvl1_assembly['value'], lvl1_assembly['sequence']))
lvl1_assembly

,variable,value,name,sequence,level
0,promoter,PBAD_P,pJ0-PBAD_P,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,level-0
1,promoter,PCin_P,pJ0-PCin_P,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,level-0
2,promoter,PSalTTC_P,pJ0-PSalTTC_P,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,level-0
3,rbs,B0034_RN,pJUMP18-B0034-MV_RN,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,addgene
4,cds,sGFP_O,pJUMP18-sfGFP_O,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,addgene
5,cds,mCherry_O,pJUMP18-mCherry_O,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,addgene
6,terminator,B0015_CT,pJUMP19-B0015_CT,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,addgene
7,vector,1A,pJUMP29-1A(sfGFP),AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,vectors


In [8]:
def assemble_lvl_1(assembly_plan, mapping):

    constructs = []
    for i, entry in assembly_plan.iterrows():

        _id = entry[0]
        name = entry[1]
        vector = entry[-1]
        parts = entry[2:-1].tolist()

        fragments = []
        fragments.append(get_sites(mapping[vector], odd_level=True, vector=True))
        for part in parts:
            fragments.append(get_sites(mapping[part], odd_level=True, vector=False))
            
        sites = list(map(list, zip(*[fragment[:2] for fragment in fragments])))
        sites[0] = sites[0][1:] + [sites[0][0]]
        
        if (sites[0]==sites[1]):
            assembly = reindex_ps1(''.join([fragment[2][:-4] for fragment in fragments]))
            constructs.append(('pJ1-' + name, assembly))
        else:
            print('Error at', name)
        
    df_constructs = pd.DataFrame(constructs, columns=['name', 'sequence'])
    df_constructs['size'] = df_constructs['sequence'].apply(lambda x: len(x))
    df_constructs['amplicon'] = df_constructs['sequence'].apply(len_amplicon)
    return df_constructs
        
assemble_lvl_1_res = assemble_lvl_1(assembly_plan, assembly_plan_map)
assemble_lvl_1_res = assemble_lvl_1_res[~assemble_lvl_1_res['name'].isin(plasmids['name'].tolist())]
assemble_lvl_1_res#.to_csv('datasets/jump/level-1-simulate.csv', index=False)

,name,sequence,size,amplicon
0,pJ1-PBAD-34-sGFP_A,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,4156,1498
1,pJ1-PCin-34-sGFP_A,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,4098,1440
2,pJ1-PSalTTC-34-sGFP_A,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,3988,1330
3,pJ1-PBAD-34-mCherry_A,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,4150,1492
4,pJ1-PCin-34-mCherry_A,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,4092,1434
5,pJ1-PSalTTC-34-mCherry_A,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,3982,1324


In [12]:
def assemble_lvl_1(assembly_plan, mapping):

    constructs = []
    for i, entry in assembly_plan.iterrows():

        _id = entry[0]
        name = entry[1]
        vector = entry[-1]
        parts = entry[2:-1].tolist()

        fragments = []
        fragments.append(get_sites(mapping[vector], odd_level=True, vector=True))
        for part in parts:
            fragments.append(get_sites(mapping[part], odd_level=True, vector=False))
            
        sites = list(map(list, zip(*[fragment[:2] for fragment in fragments])))
        
        print([fragment[:2] for fragment in fragments])
        
        sites[0] = sites[0][1:] + [sites[0][0]]
        
        print(name, sites[0], sites[1])
        
        if (sites[0]==sites[1]):
            assembly = reindex_ps1(''.join([fragment[2][:-4] for fragment in fragments]))
            constructs.append(('pJ1-' + name, assembly))
        else:
            print('Error at', name)
        
    df_constructs = pd.DataFrame(constructs, columns=['name', 'sequence'])
    df_constructs['size'] = df_constructs['sequence'].apply(lambda x: len(x))
    df_constructs['amplicon'] = df_constructs['sequence'].apply(len_amplicon)
    return df_constructs

assemble_lvl_1_res = assemble_lvl_1(assembly_plan, assembly_plan_map)
assemble_lvl_1_res = assemble_lvl_1_res[~assemble_lvl_1_res['name'].isin(plasmids['name'].tolist())]
assemble_lvl_1_res#.to_csv('datasets/jump/level-1-simulate.csv', index=False)

[('CGCT', 'GGAG'), ('GGAG', 'TACT'), ('TACT', 'AGCC'), ('AGCC', 'TTCG'), ('TTCG', 'CGCT')]
PBAD-34-sGFP_A ['GGAG', 'TACT', 'AGCC', 'TTCG', 'CGCT'] ['GGAG', 'TACT', 'AGCC', 'TTCG', 'CGCT']
[('CGCT', 'GGAG'), ('GGAG', 'TACT'), ('TACT', 'AGCC'), ('AGCC', 'TTCG'), ('TTCG', 'CGCT')]
PCin-34-sGFP_A ['GGAG', 'TACT', 'AGCC', 'TTCG', 'CGCT'] ['GGAG', 'TACT', 'AGCC', 'TTCG', 'CGCT']
[('CGCT', 'GGAG'), ('GGAG', 'TACT'), ('TACT', 'AGCC'), ('AGCC', 'TTCG'), ('TTCG', 'CGCT')]
PSalTTC-34-sGFP_A ['GGAG', 'TACT', 'AGCC', 'TTCG', 'CGCT'] ['GGAG', 'TACT', 'AGCC', 'TTCG', 'CGCT']
[('CGCT', 'GGAG'), ('GGAG', 'TACT'), ('TACT', 'AGCC'), ('AGCC', 'TTCG'), ('TTCG', 'CGCT')]
PBAD-34-mCherry_A ['GGAG', 'TACT', 'AGCC', 'TTCG', 'CGCT'] ['GGAG', 'TACT', 'AGCC', 'TTCG', 'CGCT']
[('CGCT', 'GGAG'), ('GGAG', 'TACT'), ('TACT', 'AGCC'), ('AGCC', 'TTCG'), ('TTCG', 'CGCT')]
PCin-34-mCherry_A ['GGAG', 'TACT', 'AGCC', 'TTCG', 'CGCT'] ['GGAG', 'TACT', 'AGCC', 'TTCG', 'CGCT']
[('CGCT', 'GGAG'), ('GGAG', 'TACT'), ('TACT', 'AGCC

,name,sequence,size,amplicon
0,pJ1-PBAD-34-sGFP_A,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,4156,1498
1,pJ1-PCin-34-sGFP_A,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,4098,1440
2,pJ1-PSalTTC-34-sGFP_A,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,3988,1330
3,pJ1-PBAD-34-mCherry_A,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,4150,1492
4,pJ1-PCin-34-mCherry_A,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,4092,1434
5,pJ1-PSalTTC-34-mCherry_A,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,3982,1324


In [ ]:
a = SeqRecord(Seq(assemble_lvl_1_res.iloc[0]['sequence']), id=assemble_lvl_1_res.iloc[0]['name'], description='')
print(a.format('fasta'))

In [ ]:
assemble_lvl_1_res.to_csv('datasets/jump/level-1-simulate.csv', index=False)

Level 2.

In [13]:
assembly_plan = pd.read_csv('datasets/jump/level-2-assembly.csv')
lvl2_assembly = assembly_plan[['A', 'B', 'C', 'D', 'E', 'vector']].melt()
lvl2_assembly['name'] = lvl2_assembly['value'].apply(lambda x: mapping[x] if x in mapping else '')
lvl2_assembly.loc[lvl2_assembly['name']=='', 'name'] = 'pJ1-' + lvl2_assembly['value']
lvl2_assembly = lvl2_assembly.drop_duplicates().reset_index(drop=True)
lvl2_assembly = pd.merge(lvl2_assembly, plasmids, on='name', how='left')
assembly_plan_map = dict(zip(lvl2_assembly['value'], lvl2_assembly['sequence']))

In [17]:
def assemble_lvl_2(assembly_plan, mapping):

    constructs = []
    for i, entry in assembly_plan.iterrows():

        _id = entry[0]
        name = entry[1]
        vector = entry[-1]
        parts = entry[2:-1].tolist()

        fragments = []
        fragments.append(get_sites(mapping[vector], odd_level=False, vector=True))
        for part in parts:
            fragments.append(get_sites(mapping[part], odd_level=False, vector=False))
            
        print(len(fragments))
        
        sites = list(map(list, zip(*[fragment[:2] for fragment in fragments])))
        sites[0] = sites[0][1:] + [sites[0][0]]
        
        if (sites[0]==sites[1]):
            print(sites[0], sites[1])
            assembly = reindex_ps1(''.join([fragment[2][:-4] for fragment in fragments]))
            constructs.append(('pJ2-' + name, assembly))
        else:
            print('Error at', name)
        
    df_constructs = pd.DataFrame(constructs, columns=['name', 'sequence'])
    df_constructs['size'] = df_constructs['sequence'].apply(lambda x: len(x))
    df_constructs['amplicon'] = df_constructs['sequence'].apply(len_amplicon)
    return df_constructs
        
assemble_lvl_2_res = assemble_lvl_2(assembly_plan, assembly_plan_map)
assemble_lvl_2_res = assemble_lvl_2_res[~assemble_lvl_2_res['name'].isin(plasmids['name'].tolist())]
assemble_lvl_2_res.to_csv('datasets/jump/level-2-simulate.csv', index=False)

6
['GGAG', 'AATG', 'AGCC', 'TTCG', 'GCTT', 'CGCT'] ['GGAG', 'AATG', 'AGCC', 'TTCG', 'GCTT', 'CGCT']
6
['GGAG', 'AATG', 'AGCC', 'TTCG', 'GCTT', 'CGCT'] ['GGAG', 'AATG', 'AGCC', 'TTCG', 'GCTT', 'CGCT']
6
['GGAG', 'AATG', 'AGCC', 'TTCG', 'GCTT', 'CGCT'] ['GGAG', 'AATG', 'AGCC', 'TTCG', 'GCTT', 'CGCT']
6
['GGAG', 'AATG', 'AGCC', 'TTCG', 'GCTT', 'CGCT'] ['GGAG', 'AATG', 'AGCC', 'TTCG', 'GCTT', 'CGCT']
6
['GGAG', 'AATG', 'AGCC', 'TTCG', 'GCTT', 'CGCT'] ['GGAG', 'AATG', 'AGCC', 'TTCG', 'GCTT', 'CGCT']
6
['GGAG', 'AATG', 'AGCC', 'TTCG', 'GCTT', 'CGCT'] ['GGAG', 'AATG', 'AGCC', 'TTCG', 'GCTT', 'CGCT']


In [15]:
assemble_lvl_2_res

,name,sequence,size,amplicon
0,pJ2-PBAD-PCin-PSalTTC-E20-E11_B,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,7558,4841
1,pJ2-PBAD-PCin-PSalTTC-E38-E11_B,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,7542,4825
2,pJ2-PBAD-PCin-PSalTTC-E11-E20_B,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,7558,4841
3,pJ2-PBAD-PCin-PSalTTC-E38-E20_B,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,7373,4656
4,pJ2-PBAD-PCin-PSalTTC-E11-E38_B,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,7542,4825
5,pJ2-PBAD-PCin-PSalTTC-E20-E38_B,AGGGCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACA...,7373,4656


In [ ]:
def assemble_lvl_1(assembly_plan, mapping):

    constructs = []
    for i, entry in assembly_plan.iterrows():

        _id = entry[0]
        name = entry[1]
        vector = entry[-1]
        parts = entry[2:-1].tolist()

        fragments = []
        fragments.append(get_sites(mapping[vector], odd_level=True, vector=True))
        for part in parts:
            fragments.append(get_sites(mapping[part], odd_level=True, vector=False))
            
        sites = list(map(list, zip(*[fragment[:2] for fragment in fragments])))
        sites[0] = sites[0][1:] + [sites[0][0]]
        
        if (sites[0]==sites[1]):
            assembly = reindex_ps1(''.join([fragment[2][:-4] for fragment in fragments]))
            constructs.append(('pJ1-' + name, assembly))
        else:
            print('Error at', name)
        
    df_constructs = pd.DataFrame(constructs, columns=['name', 'sequence'])
    df_constructs['size'] = df_constructs['sequence'].apply(lambda x: len(x))
    df_constructs['amplicon'] = df_constructs['sequence'].apply(len_amplicon)
    return df_constructs